# Using TorchSharp to Generate Synthetic Data for a Regression Problem

This tutorial is based on a [PyTorch example](https://jamesmccaffrey.wordpress.com/2023/06/09/using-pytorch-to-generate-synthetic-data-for-a-regression-problem/) posted by James D. McCaffrey on his blog, ported to TorchSharp.

Synthetic data sets can be very useful when evaluating and choosing a model.

Note that we're taking some shortcuts in this example -- rather than writing the data set as a text file that can be loaded from any modeling framework, we're saving the data as serialized TorchSharp tensors. Is should be straight-forward to modify the tutorial to write the data sets as text, instead.

In [ ]:
#r "nuget: TorchSharp-cpu"

open TorchSharp
open type TorchSharp.TensorExtensionMethods

#### Generative Network

Neural networks can be used to generate data as well as train. The synthetic data can then be used to evaluate different models to see how well they can copy the behavior of the network used to produce the data.

First, we will create the model that will be used to generate the synthetic data. Later, we'll construct a second model that will be trained on the data the first model generates.

In [ ]:
type Net(n_in : int) as this = 
    inherit torch.nn.Module<torch.Tensor,torch.Tensor>("Net")

    let hid1 = torch.nn.Linear(n_in, 10)
    let oupt = torch.nn.Linear(10, 1)

    do
        let lim = 0.80;
        torch.nn.init.uniform_(hid1.weight, -lim, lim) |> ignore
        torch.nn.init.uniform_(hid1.bias, -lim, lim) |> ignore
        torch.nn.init.uniform_(oupt.weight, -lim, lim) |> ignore
        torch.nn.init.uniform_(oupt.bias, -lim, lim) |> ignore
        
        this.RegisterComponents()

    override _.forward(input) = 
        use _ = torch.NewDisposeScope()
        let z = hid1.call(input).tanh_()
        let x = oupt.call(z).sigmoid_()
        x.MoveToOuterDisposeScope()

Now that we have our generative network, we can define the method to create the data set. If you compare this with the PyTorch code, you will notice that we're relying on TorchSharp to generate a whole batch of data at once, rather than looping. We're also using TorchSharp instead of Numpy for the noise-generation.

In [ ]:
let create_data_file(net: Net, n_in: int64, fileName: string, n_items: int64) =
    let x_lo = -1.0
    let x_hi = 1.0

    let one_hundredth = 0.01.ToScalar()

    let X = (x_hi - x_lo).ToScalar() * torch.rand([|n_items; n_in|]) + x_lo.ToScalar()

    use d = torch.no_grad()

    let mutable y = net.call(X)

    y <- y + torch.rand(y.shape) * one_hundredth

    y <- torch.where(y.le(torch.tensor(0.0)), y + one_hundredth * torch.randn(y.shape) + one_hundredth, y)

    X.save(fileName + ".x")
    y.save(fileName + ".y")

let load_data_file(fileName: string) = (torch.Tensor.load(fileName + ".x"), torch.Tensor.load(fileName + ".y"))

In [ ]:
let net = new Net(6)

Create the data files.

In [ ]:
create_data_file(net, 6, "train.dat", 2000);
create_data_file(net, 6, "test.dat", 400);

#### Using the Data

Load the data from files again. This is just to demonstrate how to get the data from disk.

In [ ]:
let X_train,y_train = load_data_file("train.dat")
let X_test, y_test =  load_data_file("test.dat")

Create another model class, with slightly different logic, and train it on the generated data set.

In [ ]:
type Net2(n_in : int) as this = 
    inherit torch.nn.Module<torch.Tensor,torch.Tensor>("Net2")

    let hid1 = torch.nn.Linear(n_in, 5)
    let oupt = torch.nn.Linear(5, 1)

    do
        this.RegisterComponents()

    override _.forward(input) = 
        use _ = torch.NewDisposeScope()
        let z = hid1.call(input).relu_()
        let x = oupt.call(z).sigmoid_()
        x.MoveToOuterDisposeScope()

Create an instance of the second network, choose a loss to use, and then you're ready to train it. You also need an optimizer and maybe even an LR scheduler.

In [ ]:
let model = new Net2(6)

let loss = torch.nn.MSELoss()

let learning_rate = 0.01
let optimizer = torch.optim.Rprop(model.parameters(), learning_rate)
let scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer)

A pretty standard training loop. The input is just in one batch. It ends with evaluating the trained model on the training set.

In [ ]:
printf " initial loss = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())

for epoch = 1 to 1000 do

    let output = loss.forward(model.forward(X_train), y_train)
    
    // Clear the gradients before doing the back-propagation
    model.zero_grad()

    // Do back-progatation, which computes all the gradients.
    output.backward()

    optimizer.step() |> ignore

    if epoch % 100 = 99 then
        scheduler.step()

printf " final loss   = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())


The thing we're really curious about is how the second model does on the test set, which it didn't see during training. If the loss is significantly greater than the one from the training set, we need to train more, i.e. start another epoch. If the test set loss doesn't get closer to the training set loss with more epochs, we may need more data.

In [ ]:
loss.forward(model.forward(X_test), y_test).item<float32>()

#### Splitting the Data into Batches

If we want to be a little bit more advanced, we can split the training set into batches. 

In [ ]:
let N = X_train.shape[0]/10L
let X_batch = X_train.split(N)
let y_batch = y_train.split(N)

That means modifying the training loop, too. Running multiple batches can take longer, but the model may converge quicker, so the total time before you have the desired model may still be shorter.

In [ ]:
printf " initial loss = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())

for epoch = 1 to 1000 do

    for j = 0 to X_batch.Length-1 do

        let output = loss.forward(model.forward(X_batch[j]), y_batch[j])
        
        // Clear the gradients before doing the back-propagation
        model.zero_grad()

        // Do back-progatation, which computes all the gradients.
        output.backward()

        optimizer.step() |> ignore

    scheduler.step()

printf " final loss   = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())

In [ ]:
loss.forward(model.forward(X_test), y_test).item<float32>()

#### Dataset and DataLoader

If we wanted to be really advanced, we would use TorchSharp data sets and data loaders, which would allow us to randomize the test data set between epocs (at the end of the outer training loop). Here's how we'd do that.

In [ ]:
type SyntheticDataset(fileName: string) as this = 
    inherit torch.utils.data.Dataset()

    let mutable _data:torch.Tensor = torch.Tensor.load(fileName + ".x")
    let mutable _labels:torch.Tensor = torch.Tensor.load(fileName + ".y")

    
    override _.GetTensor(index: int64) =
        let rdic = new System.Collections.Generic.Dictionary<string, torch.Tensor>()
        rdic.Add("data", _data[index])
        rdic.Add("label", _labels[index])
        rdic

    override _.Count = _data.shape[0]

The training loop gets slightly more complex with the data set.

In [ ]:
let training_data = new SyntheticDataset("train.dat")
let train = new torch.utils.data.DataLoader(training_data, 200, shuffle=true);

In [ ]:
printf " initial loss = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())

for epoch = 1 to 1000 do

    for data in train do

        let output = loss.forward(model.forward(data["data"]), data["label"])
        
        // Clear the gradients before doing the back-propagation
        model.zero_grad()

        // Do back-progatation, which computes all the gradients.
        output.backward()

        optimizer.step() |> ignore

    scheduler.step()

printf " final loss   = %s\n" (loss.forward(model.forward(X_train), y_train).item<float32>().ToString())

It's slower, and the convergence isn't that much better, but that will depend on the model used. You just have to try and try different things.

In [ ]:
loss.forward(model.forward(X_test), y_test).item<float32>()